In [1]:
import requests
import json
import csv
from bs4 import BeautifulSoup
import modules.car_func as cf
import traceback
import cloudscraper
import pandas as pd
import numpy as np
import seaborn as sns
from decimal import Decimal
from re import sub
import matplotlib.pyplot as plt
from datetime import date
import os
%matplotlib inline

In [10]:
postcode_buy="MK3 6JS" 
radius_buy=20

postcode_sell="DT2 7LU" 
radius_sell=20

min_year=2010
max_year=2015 
min_price=0
max_price=10000

#State how many brands/ models you want to show in the final graphs 
no_brands = 50
no_models = 50

import requests
import json
import csv
from bs4 import BeautifulSoup
import modules.car_func as cf
import traceback
import cloudscraper
import pandas as pd
import numpy as np
import seaborn as sns
from decimal import Decimal
from re import sub
import matplotlib.pyplot as plt
from datetime import date
import os
%matplotlib inline

location_buy = cf.retrieve_cars(
                           postcode = postcode_buy,
                           radius = radius_buy ,
                           min_year = min_year ,
                           max_year = max_year ,
                           min_price = min_price,
                           max_price = max_price,
                           location_type = 'buy')

location_sell = cf.retrieve_cars(
                           postcode = postcode_sell,
                           radius = radius_sell,
                           min_year = min_year ,
                           max_year = max_year ,
                           min_price = min_price,
                           max_price = max_price ,
                           location_type = 'sell')

location_buy.to_csv('location_buy.csv')
location_sell.to_csv('location_sell.csv')

location_buy = pd.read_csv('location_buy.csv').drop('Unnamed: 0',axis = 1)
location_sell = pd.read_csv('location_sell.csv').drop('Unnamed: 0',axis = 1)

car_match = location_buy.merge(location_sell, how = 'inner', left_on = cf.col_select(location_buy,['name','year']),
                       right_on = cf.col_select(location_sell,['name','year']))

car_match = cf.grouped_score(car_match,'index','buy_score')
car_match = cf.grouped_score(car_match,'name_buy','score_by_name')
car_match = cf.grouped_score(car_match,'brand_buy','score_by_brand')

#top 10 cars and urls
final_output = car_match.sort_values(by = 'buy_score' ,ascending= False)[['name_buy','price_buy','price_sell','postcode_buy','link_buy','postcode_sell','link_sell','mileage_buy','mileage_sell','engine_buy','engine_sell','buy_score','score_by_brand','score_by_name']]

#Create distinct file type name for each run 
today = date.today()
file_name = " ".join([today.strftime('%a') + today.strftime('%b') + str(today.strftime('%y')),postcode_buy,str(radius_buy),postcode_sell,str(radius_sell)]).replace(' ','_')

#Create a directory for the files to be stored in 
os.mkdir("runs\\" + file_name)

final_output.to_csv("runs\\" + file_name + "\\Total.csv" )

cf.graph_breakdown(car_match,'brand_buy','score_by_brand',no_brands,"runs\\" + file_name + "\\Total_brands.png")
cf.graph_breakdown(car_match,'name_buy','score_by_name',no_models,"runs\\" + file_name + "\\Total_models.png")


In [15]:
#Update the search term below to look for specific models and output to a csv 
#######################
search_term = 'focus'
#######################
final_output[final_output['name_buy'].apply(lambda x: True if search_term.lower() in x.lower() else False )]


,name_buy,price_buy,price_sell,postcode_buy,postcode_sell,mileage_buy,mileage_sell,engine_buy,engine_sell,buy_score,score_by_brand,score_by_name
2987,Ford Focus,3695,8475,MK3 6JS,DT1 3GJ,64927.0,67016.0,1.8,2.5,3552.332090,581.578867,105.778567
2130,Ford Focus,3150,5290,MK3 6JS,DT1 3GJ,39800.0,81000.0,1.6,2.0,3484.221106,581.578867,105.778567
2984,Ford Focus,3695,7100,MK3 6JS,DT1 3GJ,64927.0,49000.0,1.8,1.6,2890.948681,581.578867,105.778567
2959,Ford Focus,4694,8475,MK3 6JS,DT1 3GJ,70500.0,67016.0,2.0,2.5,2875.319104,581.578867,105.778567
2924,Ford Focus,1364,8475,MK3 6JS,DT1 3GJ,107000.0,67016.0,1.6,2.5,2850.397165,581.578867,105.778567
...,...,...,...,...,...,...,...,...,...,...,...,...
1088,Ford Focus,2495,2495,MK3 6JS,DT1 3GJ,109900.0,129000.0,1.8,1.8,NaN,581.578867,105.778567
2073,Ford Focus,2495,2495,MK3 6JS,DT1 3GJ,106000.0,119000.0,1.6,1.6,NaN,581.578867,105.778567
2081,Ford Focus,2495,2495,MK3 6JS,DT1 3GJ,119000.0,119000.0,1.6,1.6,NaN,581.578867,105.778567
2090,Ford Focus,5290,5290,MK3 6JS,DT1 3GJ,81000.0,81000.0,2.0,2.0,NaN,581.578867,105.778567


In [2]:
#No need to run just for testing purposes 
location_buy = pd.read_csv('location_buy.csv').drop('Unnamed: 0',axis = 1)
location_sell = pd.read_csv('location_sell.csv').drop('Unnamed: 0',axis = 1)

car_match = location_buy.merge(location_sell, how = 'inner', left_on = cf.col_select(location_buy,['name','year']),
                       right_on = cf.col_select(location_sell,['name','year']))

car_match = cf.grouped_score(car_match,'index','buy_score')
car_match = cf.grouped_score(car_match,'name_buy','score_by_name')
car_match = cf.grouped_score(car_match,'brand_buy','score_by_brand')